In [18]:
import pandas as pd

df = pd.read_csv('company_ceo_31-03.csv')
df

,companyLabel,ceoLabel
0,Nvidia,Jensen Huang
1,BNP Paribas,Jean-Laurent Bonnafé
2,"Block, Inc.",Jack Dorsey
3,Reliance Industries Limited,Mukesh Ambani
4,Frasers Group,Mike Ashley
...,...,...
340,Banca Transilvania,Ömer Tetik
341,Miniso,Ye Guofu
342,Deutsche Post AG,Tobias Meyer
343,Intel,Michelle Johnston Holthaus


In [ ]:
# df['rankingDate'] = pd.to_datetime(df['rankingDate']).dt.date
# df_intermediate = df.groupby(['ranking', 'playerLabel'])['rankingDate'].agg('last').reset_index()
# df_intermediate.sort_values(by=['ranking', 'rankingDate'], ascending=[True, False]).drop_duplicates(subset=['ranking'], keep='first').to_csv('query_5_result.csv', index=False)

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

serp_key = os.getenv('SERPER_API_KEY')
uni_api_key = os.getenv('UNI_API_KEY')

In [5]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    openai_api_base="https://llms-inference.innkube.fim.uni-passau.de",
    api_key=uni_api_key,
    model = "deepseekr1",
    temperature=0.6
)



/tmp/ipykernel_2284/2943525429.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(


In [6]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.utilities import GoogleSerperAPIWrapper


# Initialize search tool
search = GoogleSerperAPIWrapper()

In [ ]:
def generate_answer(query):
    
    custom_prompt = PromptTemplate.from_template(
        "You are an AI assistant. Your task is to answer the question directly using the latest information which will be provided."
        "\nNote that search results are more up-to-date and reliable than the information you have as we are in 2025."
        "\nAnswer briefly and directly, return only the answer label."
        "\nReturn a confidence score based on the provided context: high, medium or low."
        "\n The answer should be in the following format: Answer: your_answer Confidence: your_confidence_score"
        "\nUser Question: {input}"
        "\nSearch results: {search}"
    )

    # Create an LLMChain instead of an agent
    llm_chain = LLMChain(
        llm=llm,
        prompt=custom_prompt,
        verbose=True
    )
    # Get the latest factual information first
    search_result = search.results(query)
        
    formatted_results = "\n".join(
        f"Title: {res['title']}\nSnippet: {res['snippet']}\nDate: {res.get('date', 'N/A')}\n"
        for res in search_result['organic']
    )
    # Pass the query to the agent
    try:
        result = llm_chain.run({"input": query, "search": formatted_results})
        print(f"\nFinal Result: {result.strip()}")
    except Exception as e:
        print(f"An error occurred: {e}")
        
    return result.strip()
        

In [22]:
from tqdm import tqdm
tqdm.pandas()

In [25]:
df['prediction']=df['companyLabel'].progress_apply(lambda x: generate_answer(f"Who is the current CEO of {x}?"))

  0%|          | 0/345 [00:00<?, ?it/s]Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
You are an AI assistant. Your task is to answer the question directly using the latest information which will be provided.
Note that search results are more up-to-date and reliable than the information you have as we are in 2025.
Answer briefly and directly, return only the answer label.
Return a confidence score based on the provided context: high, medium or low.
User Question: Who is the current CEO of Nvidia?
Search results: Title: Jensen Huang - Wikipedia
Snippet: Jensen Huang ; Businessman; electrical engineer; philanthropist · Co-founding Nvidia · President and CEO of Nvidia Corporation (1993–present) · Lori Huang. ​. ( m.
Date: N/A

Title: Jensen Huang - NVIDIA - LinkedIn
Snippet: In 1993, I founded NVIDIA with Chris Malachowsky and Curtis Priem to solve the problem of… · Experience: NVIDIA · Education: Stanford University · Location: ...
Date: N/A

Title: Exec Bios - NVIDIA Newsroom
Snippet: Jensen Huang. Founder, President and CEO. Jensen Huang founded

  1%|          | 2/345 [00:24<1:09:33, 12.17s/it]


> Finished chain.

Final Result: <think>
Okay, so I need to figure out who the current CEO of Nvidia is. I remember hearing that Jensen Huang founded the company, but I'm not entirely sure if he's still the CEO now in 2025. Let me go through the search results provided.

Looking at the first result from Wikipedia, it says Jensen Huang has been the President and CEO since 1993, which is a long time. That gives some confidence that he's still in the role unless there's recent news of a change. The LinkedIn profile also mentions him as the co-founder and doesn't indicate any departure.

The NVIDIA Newsroom explicitly states he's the Founder, President, and CEO, serving since 1993. That seems pretty official. Forbes corroborates this information too, noting that he's been CEO since the company started. 

There are some recent dates in the snippets as well. A YouTube video from January 27, 2025, talks about his vision for the future, which suggests he was active as CEO at least up until th

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


...


> Finished chain.

Final Result: <think>
Alright, let's figure out who the current CEO of 21st Century Fox is. I'm a bit confused because the search results have different information. 

First, I see that Lachlan Murdoch is the Executive Chair and CEO of Fox Corporation in some snippets. But wait, isn't Fox Corporation different from 21st Century Fox? I remember that Disney acquired most of 21st Century Fox, so maybe there's a spin-off or rebranding involved here.

Looking at another result, James Murdoch was the CEO of 21st Century Fox before, but he left. There's a snippet saying he became CEO starting July 1, 2015, but that might be outdated because it's from Jun 16, 2015. 

Another search result mentions that Lachlan Murdoch is the co-chairman of 21st Century Fox, along with his brother James. But now, in Sep 2023, he's the CEO of Fox Corporation. I think after Disney bought 21st Century Fox, the remaining part became Fox Corporation, so maybe Lachlan is leading that.

Wait, but s

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
You are an AI assistant. Your task is to answer the question directly using the latest information which will be provided.
Note that search results are more up-to-date and reliable than the information you have as we are in 2025.
Answer briefly and directly, return only the answer label.
Return a confidence score based on the provided context: high, medium or low.
User Question: Who is the current CEO of Sears?
Search results: Title: Eddie Lampert - Wikipedia
Snippet: Edward Scott Lampert (born July 19, 1962) is an American billionaire businessman. He is the former chief executive and chairman of Sears Holdings, the founder ...
Date: N/A

Title: Sears Executive Team - Comparably
Snippet: Eddie Lampert serves as the CEO / President of Sears. ... Mr. Michael A. Miles was a Special Limited Partner at Forstmann Little & Co. Mr. Miles served as the ...
Date: N/A

Title: Sears - Wikipedia
Snippet: In May 2019, former Sears Holdings chairman and CEO Eddie Lampert, mon

  4%|▍         | 15/345 [06:21<2:19:44, 25.41s/it]


KeyboardInterrupt: 

In [16]:

custom_prompt = PromptTemplate.from_template(
    "You are an AI assistant. Your task is to answer the question directly using the latest information which will be provided."
    "\nNote that search results are more up-to-date and reliable than the information you have as we are in 2025."
    "\nAnswer briefly and directly, return only the answer label."
    "\nReturn a confidence score based on the provided context: high, medium or low."
    "\nUser Question: {input}"
    "\nSearch results: {search}"
)

# Create an LLMChain instead of an agent
llm_chain = LLMChain(
    llm=llm,
    prompt=custom_prompt,
    verbose=True
)

# Query
#query = "Who is the current president/ leader of the organization: All European Academies?"
query = "Who is the current CEO of the company: Intel?"

# Get the latest factual information first
search_result = search.results(query)

# if search_result:
#     print(f"Raw Search API Response: {search_result}\n")
# else:
#     print("Error: Web Search API returned None or an invalid response.")
    
formatted_results = "\n".join(
    f"Title: {res['title']}\nSnippet: {res['snippet']}\nDate: {res.get('date', 'N/A')}\n"
    for res in search_result['organic']
)

# Pass the query to the agent
try:
    result = llm_chain.run({"input": query, "search": formatted_results})
    print(f"\nFinal Result: {result.strip()}")
except Exception as e:
    print(f"An error occurred: {e}")



Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
You are an AI assistant. Your task is to answer the question directly using the latest information which will be provided.
Note that search results are more up-to-date and reliable than the information you have as we are in 2025.
Answer briefly and directly, return only the answer label.
Return a confidence score based on the provided context: high, medium or low.
User Question: Who is the current CEO of the company: Intel?
Search results: Title: Intel Appoints Lip-Bu Tan as Chief Executive Officer - Newsroom
Snippet: Lip-Bu Tan was appointed chief executive officer of Intel Corporation in March 2025. He also serves on the company's board of directors.
Date: Mar 12, 2025

Title: Intel's new CEO tells customers 'be brutally honest with us' | Reuters
Snippet: In his first remarks as Intel's new CEO, Lip-Bu Tan on Monday outlined a leaner version of the iconic American chipmaker in which he would ...
Date: 15 hours ago

Title: Intel Appoints Lip-Bu Tan as Chief Ex

In [7]:
# Query

#query = "Who is the current president/ leader of the organization: All European Academies?"
#query = "All european Academies current President/ Leader?"
query = "Who is the current CEO of the company: Reliance Industries Limited	"
print(f"Query: {query}\n")

# Get the latest factual information first
search_result = search.results(query)
search_result

Query: Who is the current CEO of the company: Reliance Industries Limited	



{'searchParameters': {'q': 'Who is the current CEO of the company: Reliance Industries Limited\t',
  'gl': 'us',
  'hl': 'en',
  'type': 'search',
  'num': 10,
  'engine': 'google'},
 'answerBox': {'snippet': 'Mukesh Dhirubhai Ambani (born 19 April 1957) is an Indian billionaire and businessman who is the chairman and managing director of Reliance Industries.',
  'snippetHighlighted': ['Mukesh Dhirubhai Ambani'],
  'title': 'Mukesh Ambani - Wikipedia',
  'link': 'https://en.wikipedia.org/wiki/Mukesh_Ambani'},
 'organic': [{'title': 'Reliance Industries Limited Chairman & Managing Director',
   'link': 'https://www.ril.com/about/chairman-managing-director',
   'snippet': 'RIL Chairman and Managing Director, Mukesh D. Ambani, is a chemical engineer from the Institute of Chemical Technology, Mumbai (erstwhile the University ...',
   'position': 1},
  {'title': 'Reliance Industries Limited Board of Directors - Nita Ambani',
   'link': 'https://www.ril.com/about/board-of-directors',
   'sni

In [2]:
import pandas as pd
resultant_dataframe = pd.read_csv("company_ceo_results.csv")

In [3]:
resultant_dataframe['confidence'] = resultant_dataframe['prediction'].str.extract(r'(?i)\b(high|medium|low)\b')

In [4]:
import re

def extract_explanation(text):
    match = re.search(r'<think>(.*?)</think>', text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

In [5]:
resultant_dataframe['expalanation'] = resultant_dataframe['prediction'].apply(extract_explanation)

In [6]:
extract_explanation(resultant_dataframe['prediction'].iloc[0])

"Okay, so I need to figure out who the current CEO of Nvidia is. Let me go through each search result carefully.\n\nThe first result from Wikipedia says Jensen Huang co-founded Nvidia and has been President and CEO since 1993. That seems pretty solid. The second result from LinkedIn also mentions he founded Nvidia in 1993, so that's consistent.\n\nLooking at the NVIDIA Newsroom snippet, it again states he's the Founder, President, and CEO since the company started. Forbes echoes this information as well, adding that he still owns a percentage of the company. \n\nThere are some YouTube videos mentioning him as the current CEO in 2025 and 2024. A news article from February 26, 2025, refers to him as the CEO during a conference. Another video from March 1, 2024, also interviews him as the CEO.\n\nThe Reddit snippet talks about his role as CEO since April 1993 when he co-founded Nvidia with others. All these sources consistently mention Jensen Huang as the CEO without any indication of a c

In [7]:
resultant_dataframe['explanation']= resultant_dataframe['prediction'].apply(extract_explanation)

In [8]:
import re


def retrieve_answer_span(text):
    pattern = r"</think>\s*(.*)"
    match = re.search(pattern, text, re.DOTALL)
    result = match.group(1) if match else None
    return result


In [9]:
resultant_dataframe['pre_answer'] = resultant_dataframe['prediction'].apply(retrieve_answer_span)

In [51]:
resultant_dataframe['pre_answer']

0      The current CEO of Nvidia is Jensen Huang.\n\n...
1      Jean-Laurent Bonnafé is the current CEO of BNP...
2      Jack Dorsey is the current CEO of Block, Inc.\...
3      Mukesh D. Ambani is the current CEO of Relianc...
4      The current CEO of Frasers Group is Michael Mu...
                             ...                        
340                                           Ömer Tetik
341    Ye Guofu is the current CEO of Miniso.\n\nAnsw...
342    Dr. Tobias Meyer is the current CEO of Deutsch...
343    The current CEO of Intel is Lip-Bu Tan.  \nCon...
344    The current CEO of Commonwealth Bank is Matt C...
Name: pre_answer, Length: 345, dtype: object

In [13]:
def extract_answer(text):
    pattern=r"(?:Answer:\s*[**]*\s*|\*\*Answer:\*\*\s*|The answer is:\s*|The answer label is: |The current CEO of .*? is \*\*|The current .*? is [**]*)\s*(?:<[^>]+>\s*)*([\w\s,.'-]+?)(?=\s*(?:\*\*\s*\.\s*\n\n|Confidence|$))"
    #pattern=r"(?:Answer:\s*[**]*\s*|The answer is:\s*|The current .*? is [**]*|</think>\s*\n+\s*)\s*(?:<[^>]+>\s*)*([\w\s,.-]+?)(?=\s*(?:\*\*\s*\.\s*\n\n|Confidence|$))"
   
    matches = re.findall(pattern, text, re.IGNORECASE | re.MULTILINE)
    last_match = matches[-1] if matches else None
    return last_match


In [14]:
resultant_dataframe['answer'] = resultant_dataframe['pre_answer'].apply(extract_answer)

In [141]:
resultant_dataframe['answer']

0              Jensen Huang
1      Jean-Laurent Bonnafé
2               Jack Dorsey
3          Mukesh D. Ambani
4            Michael Murray
               ...         
340                    None
341                Ye Guofu
342        Dr. Tobias Meyer
343             Lip-Bu Tan.
344              Matt Comyn
Name: answer, Length: 345, dtype: object

In [ ]:
import re

    
def second_retrieve(text):
    pattern=r"^([A-Za-zÀ-ÖØ-öø-ÿ]+(?: [A-Za-zÀ-ÖØ-öø-ÿ\.\-]+)*?)(?=\s*(?:\n|Confidence|is the current|$))"
    #pattern = r'''(?:\*\*([^*]+)\*\*|^([A-Za-zÀ-ÖØ-öø-ÿ]+(?:\s[A-Za-zÀ-ÖØ-öø-ÿ.\-]+)*))(?=\s*(?:\n|Confidence|is the current|$))'''
    match = re.search(pattern, text, re.IGNORECASE | re.MULTILINE)

    if match:
        return match.group(1)
    return None
    

In [19]:
resultant_dataframe['pre_answer'].iloc[133]

'The current CEO of Norwegian Air Shuttle is **Geir Karlsen**.  \nConfidence score: **high**'

In [18]:
interm = resultant_dataframe[resultant_dataframe['answer'].isnull()]
interm['answer']= interm['pre_answer'].apply(second_retrieve)
interm[interm['answer'].isnull()]

/tmp/ipykernel_648/2482192990.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interm['answer']= interm['pre_answer'].apply(second_retrieve)


,Unnamed: 0,companyLabel,ceoLabel,prediction,confidence,expalanation,explanation,pre_answer,answer
29,29,Q9265848,Krzysztof Brzeziński,"<think>\nOkay, so I need to figure out who the...",Low,"Okay, so I need to figure out who the current ...","Okay, so I need to figure out who the current ...",The current CEO of Q9265848 cannot be determin...,None
50,50,Q3077586,Chen Rui,"<think>\nOkay, so I need to figure out who the...",low,"Okay, so I need to figure out who the current ...","Okay, so I need to figure out who the current ...",The current CEO of Q3077586 cannot be determin...,None
79,79,Songa Offshore SE,Bjørnar Iversen,"<think>\nAlright, let's try to figure out who ...",medium,"Alright, let's try to figure out who the curre...","Alright, let's try to figure out who the curre...",The most recent information indicates that aft...,None
133,133,Norwegian Air Shuttle,Geir Karlsen,"<think>\nOkay, I need to figure out who the cu...",high,"Okay, I need to figure out who the current CEO...","Okay, I need to figure out who the current CEO...",The current CEO of Norwegian Air Shuttle is **...,None
143,143,Fortescue Metals Group,Elizabeth Gaines,"<think>\nOkay, I need to figure out who the cu...",High,"Okay, I need to figure out who the current CEO...","Okay, I need to figure out who the current CEO...",The current CEO of Fortescue Metals Group is *...,None
147,147,HP Inc.,Dion Weisler,"<think>\nOkay, I need to figure out who the cu...",high,"Okay, I need to figure out who the current CEO...","Okay, I need to figure out who the current CEO...",The current CEO of HP Inc. is **Enrique Lores*...,None
149,149,Caterpillar Inc.,Jim Umpleby,"<think>\nOkay, so I need to figure out who the...",High,"Okay, so I need to figure out who the current ...","Okay, so I need to figure out who the current ...",The current CEO of Caterpillar Inc. is D. Jame...,None
151,151,Orange Polska,Bruno Duthoit,"<think>\nOkay, so I need to figure out who the...",High,"Okay, so I need to figure out who the current ...","Okay, so I need to figure out who the current ...",The current CEO of Orange Polska is **Liudmila...,None
157,157,Alphabet Inc.,Sundar Pichai,"<think>\nOkay, so I need to figure out who the...",high,"Okay, so I need to figure out who the current ...","Okay, so I need to figure out who the current ...",The current CEO of Alphabet Inc. is **Sundar P...,None
159,159,ADA,Nicolas Rousselet,"<think>\nOkay, I need to figure out who the cu...",High,"Okay, I need to figure out who the current CEO...","Okay, I need to figure out who the current CEO...","Charles D. ""Chuck"" Henderson is the current CE...",None


In [122]:
resultant_dataframe[resultant_dataframe['answer'].isnull()]['pre_answer'].apply(second_retrieve)

21                 Elad Aharonson
24              Charles W. Scharf
29                           None
34                  Peter Zaffino
43              Damon T. Hininger
                  ...            
325               Helen Lofthouse
333            Ignacio Madridejos
335                 Jaime Ramirez
336    Juan López-Belmonte Encina
340                    Ömer Tetik
Name: pre_answer, Length: 73, dtype: object

In [265]:
resultant_dataframe[resultant_dataframe['ceoLabel']!=resultant_dataframe['answer']]

,Unnamed: 0,companyLabel,ceoLabel,prediction,confidence,explanation,answer
0,0,Nvidia,Jensen Huang,"<think>\nOkay, so I need to figure out who the...",High,"Okay, so I need to figure out who the current ...",Jensen Huang.
3,3,Reliance Industries Limited,Mukesh Ambani,"<think>\nOkay, I need to figure out who the cu...",High,"Okay, I need to figure out who the current CEO...",Mukesh D. Ambani
4,4,Frasers Group,Mike Ashley,"<think>\nOkay, I need to figure out who the cu...",High,"Okay, I need to figure out who the current CEO...",correct.
5,5,Lazard,Peter R. Orszag,"<think>\nAlright, so I need to figure out who ...",High,"Alright, so I need to figure out who the curre...",Peter Orszag
7,7,Lenovo,Yang Yuanqing,"<think>\nOkay, so I need to figure out who the...",High,"Okay, so I need to figure out who the current ...",Yuanqing Yang.
...,...,...,...,...,...,...,...
339,339,Tessellis,Davide Rota,"<think>\nOkay, I need to figure out who the cu...",High,"Okay, I need to figure out who the current CEO...","outdated since it refers to the founder, which..."
340,340,Banca Transilvania,Ömer Tetik,"<think>\nAlright, so I need to figure out who ...",NaN,"Alright, so I need to figure out who the curre...",None
342,342,Deutsche Post AG,Tobias Meyer,"<think>\nOkay, so I need to figure out who the...",high,"Okay, so I need to figure out who the current ...",Dr. Tobias Meyer.
343,343,Intel,Michelle Johnston Holthaus,"<think>\nOkay, I need to figure out who the cu...",High,"Okay, I need to figure out who the current CEO...",based on the provided search results. Let me g...
